<a href="https://colab.research.google.com/github/kaispace30098/SKU_mapper/blob/main/Serial_number_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New PO: Serial number Generator  

In [ ]:
#@title  1.Call backend - Google Sheet


from google.colab import auth
auth.authenticate_user()
import gspread
import gspread_dataframe as gd
from oauth2client.client import GoogleCredentials
from gspread_dataframe import set_with_dataframe
import numpy as np
import datetime as dt
from datetime import datetime
import pytz
import pandas as pd
from google.colab import files

def read_dir_file(case_f):  # case_f = 0 for uploading one File and case_f = 1 for uploading one Zipped Directory
        #uploaded = files.upload()    # to upload a Full Directory, please Zip it first (use WinZip)
        for fn in uploaded.keys():
            name = fn  #.encode('utf-8')
            #print('\nfile after encode', name)
            #name = io.BytesIO(uploaded[name])
        if case_f == 0:    # case of uploading 'One File only'
            print('\n file name: ', name)
            return name
        else:   # case of uploading a directory and its subdirectories and files
            zfile = zipfile.ZipFile(name, 'r')   # unzip the directory 
            zfile.extractall()
            for d in zfile.namelist():   # d = directory
                print('\n main directory name: ', d)
                return d

gc = gspread.authorize(GoogleCredentials.get_application_default())
wb= gc.open_by_url('https://docs.google.com/spreadsheets/d/1gUYtvLPslOD7XKBzJvNnFw28uSxFiPCwzrv4JYRV6HE/edit#gid=1520865025')


sheet = wb.worksheet('PO')
#sheet2 = wb.worksheet('All PO')
data = sheet.get_all_values()
df0=pd.DataFrame(data)
df0.columns = df0.iloc[0]
df0 = df0.iloc[1:]
# using dictionary to convert specific columns  
convert_dict = {'SN_min': int, 'SN_max': int
               }  

df0 = df0.astype(convert_dict)

df0.index=df0.index-1

In [ ]:

#@title 2.Fill PO info and download the sheet for Order detail

tz_AZ = pytz.timezone('US/Arizona') 
datetime_AZ = datetime.now(tz_AZ)
current_time = datetime_AZ.strftime("%m/%d/%Y, %H:%M:%S")
#C1
PO = "3CCXJ49T" #@param {type:"string"}
Vendor = "NINGBO CARELINE ELECTRIC APPLIANCE CO.,LT" #@param {type:"string"}
Purchase_Date = "2022-01-17" #@param {type:"date"}
Expected_Date = "2022-04-24" #@param {type:"date"}
##C2 DOWNLOAD the PO sheet
df_temp=[]
df_temp = pd.DataFrame (columns = ['SKU','Quantity'])
df_temp.to_csv('purchase template.csv',index=False) 
files.download('purchase template.csv')
#下載範例表單(由Google colab去call google sheet API)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title 3. Upload Order detail 
#C3 UPLOAD PO sheet
uploaded = files.upload()
fileName = read_dir_file(0)
import io
df_c= pd.read_csv(io.BytesIO(uploaded[fileName]))# Dataset is now stored in a Pandas Dataframe
df_c['PO']=PO
df_c['Vendor']=Vendor
df_c['Expected_Date']=Expected_Date
df_c['Purchased_Date']=Purchase_Date
df_c['Last_Update_Date']=Purchase_Date
df_c["SN_min"]=""
df_c["SN_max"]=""
df_c["Last_Update_Time"]=current_time
#-------------------------------------------------------------------------------------------------------------------------
base = dt.datetime.today().date()
todaybase=int(repr(base.year)[-2])*100000000000+int(repr(base.year)[-1])*10000000000+int(base.month)*100000000+int(base.day)*1000000
#F
x = range(1,df_c.shape[0])
if todaybase > df0['SN_max'].max():
  df_c['SN_min'][0]=todaybase+1
  df_c['SN_max'][0]=df_c['SN_min'][0]+ df_c['Quantity'][0]-1
  for n in x:
    df_c['SN_min'][n]=df_c['SN_max'][n-1]+1
    df_c['SN_max'][n]=df_c['SN_max'][n-1]+ df_c['Quantity'][n]
else:
  df_c['SN_min'][0]=df0['SN_max'].max()+1
  df_c['SN_max'][0]=df_c['SN_min'][0]+df_c['Quantity'][0]-1
  for n in x:
    df_c['SN_min'][n]=df_c['SN_max'][n-1]+1
    df_c['SN_max'][n]=df_c['SN_max'][n-1]+ df_c['Quantity'][n]
frames = [df0, df_c]
result = pd.concat(frames)
gd.set_with_dataframe(sheet, result)

#df_cc=df[........]
#frame2=[df1,df_cc]
#result2 = pd.concat(frames2)
#gd.set_with_dataframe(sheet2, result)



In [ ]:
#@title Query:serial number lookup

Serial_number = 210205000001 #@param {type:"number"}

for i in range(0, len(result)):
    if Serial_number >= int(result.iloc[i]['SN_min'])and Serial_number <= int(result.iloc[i]['SN_max']):
      print(result.iloc[i]['PO'])

